<div style="text-align: center;">
    <img src="img/logo.png" width="400" alt="logo_uba">
</div>

**Alumnos:** <br />
Pablo Martin Gomez Verdini - gomezpablo86@gmail.com <br />
Diego Paciotti Iacchelli - diegopaciotti@gmail.com  <br />
Joaquin Gonzalez - joagonzalez@gmail.com <br />

**Repositorio Github**<br />
https://github.com/FIUBA-CEIA-18Co2024/IIA-TP2 <br />


In [1]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
print(housing.data.shape, housing.target.shape)
print(housing.feature_names[0:6])